<a target="_blank" href="https://colab.research.google.com/github/segmed/gemma_mistral/blob/main/notebooks/datasets_example_results.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!nvidia-smi

Sun Feb 25 17:10:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# if you are using google colab

import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
if os.environ["HF_TOKEN"] == None:
    from huggingface_hub import notebook_login
    notebook_login()

In [3]:
!pip install -qq "transformers==4.38.0" --upgrade
!pip -qq install bitsandbytes accelerate datasets pandas peft trl tqdm evaluate
!pip -qq install "torch>=2.1.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [4]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42).train_test_split(test_size=0.01)
dataset

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 180993
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 1829
    })
})

In [5]:
import pandas as pd
df_test = pd.DataFrame(dataset['test'])
df_test.head(1)

,id,question,opa,opb,opc,opd,cop,choice_type,exp,subject_name,topic_name
0,732974dc-af19-4d8b-9c2d-fb4e82528356,Corrosion of Silver cones is due to presence of,Cobalt,Tin,Nickel,zinc,2,single,None,Dental,None


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "matthewchung74/gemma-mcqa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
import numpy as np

choice_map = np.array(["(a)", "(b)", "(c)", "(d)"])

def get_content(row):
    question = row['question']
    cop = row['cop']
    opa = row['opa']
    opb = row['opb']
    opc = row['opc']
    opd = row['opd']
    exp = row['exp']
    topic_name = row['topic_name']
    optiona = f"(a) {opa}"
    optionb = f"(b) {opb}"
    optionc = f"(c) {opc}"
    optiond = f"(d) {opd}"
    exp = row['exp']
    answer = choice_map[cop]

    system_prompt_choice = """Analyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:"""
    return f"{system_prompt_choice}\n{question}\n{optiona} {optionb} {optionc} {optiond}", answer, exp

def get_test_sample(row):
    content, answer, exp = get_content(row)
    chat = [
        { "role": "user", "content": content },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, add_special_tokens=False)
    return prompt

def get_test_sample_answer(row):
    content, answer, exp = get_content(row)
    return answer, exp

def get_training_sample(row):
    content, answer, exp = get_content(row)
    chat = [
        { "role": "user", "content": content },
        { "role": "model", "content": f"{answer}\n{exp}" },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False, add_special_tokens=False)
    return prompt

In [8]:
df_test['text'] = df_test.apply(get_test_sample, axis=1)

In [9]:
def predict(row, model, tokenizer):
    y_pred = None
    prompt = get_test_sample(row)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=64, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.01, eos_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(outputs[0])
    answer = result.split("<start_of_turn>model")[-1].lower()
    y_pred = "none"
    for index, choice in enumerate(choice_map):
        if choice in answer:
            y_pred = choice

    return y_pred, input_ids['input_ids'].shape[1], answer

In [10]:
for index, row in df_test.loc[3:6].iterrows():
    choice, explanation = get_test_sample_answer(row)
    prediction, num_tokens, answer = predict(row, model, tokenizer)
    display(f"prediction - {prediction}, reference {choice}")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


'prediction - (c), reference (c)'

'prediction - (a), reference (a)'

'prediction - (a), reference (a)'

'prediction - (a), reference (d)'

In [11]:
from tqdm import tqdm
import time

references = []
predictions = []
times = []
num_tokens = []
goto = 100

# for index, row in tqdm(df_test.loc[:100].iterrows(), total=df_test.shape[0]):
for index, row in tqdm(df_test.loc[:goto].iterrows(), total=goto):
    seconds = time.time()
    choice, explanation = get_test_sample_answer(row)
    reference_index = np.where(choice_map == choice)[0][0]
    references.append(reference_index)
    prediction, num_token, answer = predict(row, model, tokenizer)
    prediction_index = np.where(choice_map == prediction)[0][0]
    predictions.append(prediction_index)
    times.append(time.time() - seconds)
    num_tokens.append(num_token)

101it [13:03,  7.76s/it]


In [12]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(predictions=predictions, references=references)
display("accuracy:", results["accuracy"])

'accuracy:'

0.5247524752475248

In [13]:
average = sum(times) / len(times)
display("duration per sample:", round(average, 2))

'duration per sample:'

7.76

In [14]:
average = sum(num_tokens) / len(num_tokens)
display("avg tokens per sample:", round(average, 2))

'avg tokens per sample:'

97.43

In [15]:
import torch
torch.cuda.empty_cache()
del model
del tokenizer

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "matthewchung74/mistral-mcqa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [17]:
import numpy as np

choice_map = np.array(["(a)", "(b)", "(c)", "(d)"])

def get_content(row):
    question = row['question']
    cop = row['cop']
    opa = row['opa']
    opb = row['opb']
    opc = row['opc']
    opd = row['opd']
    exp = row['exp']
    topic_name = row['topic_name']
    optiona = f"(a) {opa}"
    optionb = f"(b) {opb}"
    optionc = f"(c) {opc}"
    optiond = f"(d) {opd}"
    exp = row['exp']
    answer = choice_map[cop]

    system_prompt_choice = """Analyze the question and determine which choice is the correct one and return the answer as one or more of the following "(a)", "(b)", "(c)", "(d)" followed by an explanation:"""
    return f"{system_prompt_choice}\n{question}\n{optiona} {optionb} {optionc} {optiond}", answer, exp

def get_test_sample(row):
    content, answer, exp = get_content(row)
    chat = [
        { "role": "user", "content": content },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, add_special_tokens=False)
    return prompt

def get_test_sample_answer(row):
    content, answer, exp = get_content(row)
    return answer, exp

def get_training_sample(row):
    content, answer, exp = get_content(row)
    chat = [
        { "role": "user", "content": content },
        { "role": "assistant", "content": f"{answer}\n{exp}" },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False, add_special_tokens=False)
    return prompt

In [20]:
def predict(row, model, tokenizer):
    y_pred = None
    prompt = get_test_sample(row)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=64, do_sample=True, top_k=0, num_return_sequences=1, temperature=0.01, eos_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(outputs[0])
    answer = result.split("[/INST]")[-1].lower()
    y_pred = "none"
    answer = answer.split()[0]

    for index, choice in enumerate(choice_map):
        # print(answer)
        # print(choice)
        if answer in choice:
            y_pred = choice

    return y_pred, input_ids['input_ids'].shape[1], answer

In [22]:
# TODO, getting warnings due to https://stackoverflow.com/questions/77301266/input-type-into-linear4bit-is-torch-float16-but-bnb-4bit-compute-type-torch-flo
from tqdm import tqdm
import time

references = []
predictions = []
times = []
num_tokens = []
goto = 100

# for index, row in tqdm(df_test.loc[:100].iterrows(), total=df_test.shape[0]):
for index, row in tqdm(df_test.loc[:goto].iterrows(), total=goto):
    seconds = time.time()
    choice, explanation = get_test_sample_answer(row)
    reference_index = np.where(choice_map == choice)[0][0]
    references.append(reference_index)
    prediction, num_token, answer = predict(row, model, tokenizer)
    if prediction == "none":
        prediction_index = -1
    else:
        prediction_index = np.where(choice_map == prediction)[0][0]
    # print(f"{reference_index} , {prediction_index}")
    predictions.append(prediction_index)
    times.append(time.time() - seconds)
    num_tokens.append(num_token)

100%|██████████| 100/100 [10:20<00:00,  6.16s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
101it [10:26,  6.21s/it]


In [23]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(predictions=predictions, references=references)
display("accuracy:", results["accuracy"])

'accuracy:'

0.12871287128712872

In [24]:
average = sum(times) / len(times)
display("duration per sample:", round(average, 2))

'duration per sample:'

6.21

In [25]:
average = sum(num_tokens) / len(num_tokens)
display("avg tokens per sample:", round(average, 2))

'avg tokens per sample:'

108.66

In [ ]:
from google.colab import runtime
runtime.unassign()